In [1]:
!nvidia-smi

Thu Dec 23 13:38:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.67       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:65:00.0 Off |                  N/A |
| 65%   59C    P2   158W / 370W |   1507MiB / 24265MiB |     85%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    Off  | 00000000:B3:00.0 Off |                  N/A |
| 74%   

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform as st
import os
import gc
import warnings
from sklearn.metrics import classification_report, average_precision_score
from PIL import Image
from Parse_TFrecords import *
from define_model import *
from load_data import *
from utilities import *
 
print(tf.__version__)

warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

gpus = tf.config.list_physical_devices(device_type='GPU')
tf.config.set_visible_devices(devices=gpus[0], device_type='GPU')

2.5.0


In [3]:
pos = 27297
neg = 119133

total = pos+neg

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.61
Weight for class 1: 2.68


# Segmentation images

In [4]:
import cv2 as cv
def coordinate(image):
    img = np.array(image, np.float32)
    img_zero = np.zeros((img.shape[0], img.shape[1]))

    kernel = np.ones((5,5), np.uint8)
    erosion = cv.erode(img, kernel, iterations = 1)
    dilation = cv.dilate(erosion, kernel, iterations = 1)
    plt.imshow(dilation)
    x_array = []
    y_array = []
    for x in range(img.shape[0]):
            flag = 0
            for y in range(img.shape[1]):
                if (dilation[y][x] > 0 and flag == 0):
                    flag = 1
                    img_zero[y][x] = 100
                    x_array.append(x)

    for y in range(img.shape[1]):
        flag = 0
        for x in range(img.shape[0]):
                if (dilation[y][x] > 0 and flag == 0):
                    flag = 1
                    img_zero[y][x] = 100
                    y_array.append(y)
    img_zero = np.zeros((img.shape[0], img.shape[1]))
    try:
        bot_right_x = max(x_array)
        bot_right_y = max(y_array)
        top_left_x = min(x_array)
        top_left_y = min(y_array)
    except:
        return [0, 0, 0, 0]

    # di = dilation[top_left_y:bot_right_y+1, top_left_x:bot_right_x+1]
    # di = st.resize(di, (256, 256))
    # plt.imshow(di)
    return [top_left_y, bot_right_y+1, top_left_x, bot_right_x+1]

In [ ]:
# Load images and retrieve the coordinator contained lungs
subject_ids = []
study_ids = []
coordinates = []
filename = ['../AI-Venger/data/segmentation_train.tfrecords']
raw_dataset = tf.data.TFRecordDataset(filename)
for raw_record in raw_dataset:
    #loads data  from the trecord all info 
    example = tf.train.Example() # subject id and study_id (contains several chest Xrays) 
    example.ParseFromString(raw_record.numpy()) #reads the example 

    subject_id = example.features.feature['subject_id'].int64_list.value[0] # change subject_id to type int
    study_id = example.features.feature['study_id'].int64_list.value[0]
    img_bytes = np.fromstring(example.features.feature['lung_image'].bytes_list.value[0], np.uint8)
    img_np = cv.imdecode(img_bytes, cv.IMREAD_GRAYSCALE)
    img_np = Image.fromarray(img_np).convert("P")
    coordinates.append(coordinate(img_np))
    subject_ids.append(subject_id)
    study_ids.append(study_id)

In [ ]:
import pandas as pd
d = {'subject_id': subject_ids, 'study_id': study_ids, 'coordinate': coordinates}
df = pd.DataFrame(data=d)
df

In [ ]:
df.to_csv(index=False, path_or_buf='train_coordinate')

In [ ]:
# Load images and retrieve the coordinator contained lungs
subject_ids = []
study_ids = []
coordinates = []
filename = ['../AI-Venger/data/segmentation_test.tfrecords']
raw_dataset = tf.data.TFRecordDataset(filename)
for raw_record in raw_dataset:
    #loads data  from the trecord all info 
    example = tf.train.Example() # subject id and study_id (contains several chest Xrays) 
    example.ParseFromString(raw_record.numpy()) #reads the example 

    subject_id = example.features.feature['subject_id'].int64_list.value[0] # change subject_id to type int
    study_id = example.features.feature['study_id'].int64_list.value[0]
    img_bytes = np.fromstring(example.features.feature['lung_image'].bytes_list.value[0], np.uint8)
    img_np = cv.imdecode(img_bytes, cv.IMREAD_GRAYSCALE)
    img_np = Image.fromarray(img_np).convert("P")
    coordinates.append(coordinate(img_np))
    subject_ids.append(subject_id)
    study_ids.append(study_id)
d = {'subject_id': subject_ids, 'study_id': study_ids, 'coordinate': coordinates}
df = pd.DataFrame(data=d)
df.to_csv(index=False, path_or_buf='test_coordinate')

In [ ]:
# Load images and retrieve the coordinator contained lungs
subject_ids = []
study_ids = []
coordinates = []
filename = ['../AI-Venger/data/segmentation_val.tfrecords']
raw_dataset = tf.data.TFRecordDataset(filename)
for raw_record in raw_dataset:
    #loads data  from the trecord all info 
    example = tf.train.Example() # subject id and study_id (contains several chest Xrays) 
    example.ParseFromString(raw_record.numpy()) #reads the example 

    subject_id = example.features.feature['subject_id'].int64_list.value[0] # change subject_id to type int
    study_id = example.features.feature['study_id'].int64_list.value[0]
    img_bytes = np.fromstring(example.features.feature['lung_image'].bytes_list.value[0], np.uint8)
    img_np = cv.imdecode(img_bytes, cv.IMREAD_GRAYSCALE)
    img_np = Image.fromarray(img_np).convert("P")
    coordinates.append(coordinate(img_np))
    subject_ids.append(subject_id)
    study_ids.append(study_id)
d = {'subject_id': subject_ids, 'study_id': study_ids, 'coordinate': coordinates}
df = pd.DataFrame(data=d)
df.to_csv(index=False, path_or_buf='val_coordinate')

In [14]:
df0 = pd.read_csv('test_coordinate')
df1 = pd.read_csv('val_coordinate')
df2 = pd.read_csv('train_coordinate')

df = pd.concat([df0, df1, df2], ignore_index=True)

df.to_csv('seg_coordinate')

## train

In [4]:
checkpoint_filepath = 'checkpoints/AUC/checkpoint_seg_Dnet121'
monitor_ = 'val_auc'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    save_weights_only=True,
    monitor=monitor_,
    mode='max')

callback = [tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.EarlyStopping(mode='max', patience=3, monitor=monitor_),
            model_checkpoint_callback]

In [5]:
BATCH_SIZE = 32

record_file_train = 'copd_train_seg_new.tfrecords'
train_dataset = (tf.data.TFRecordDataset(
    record_file_train, buffer_size=BATCH_SIZE*10, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_train)
.shuffle(total)
.batch(BATCH_SIZE))

record_file_val = 'copd_val_seg_new.tfrecords'
val_dataset = (tf.data.TFRecordDataset(
    record_file_val, buffer_size=BATCH_SIZE*10, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_train)
.shuffle(36601)
.batch(BATCH_SIZE))

In [6]:
model = load_model_from_pretrain('Dnet121')

model.summary()

metric = tf.keras.metrics.AUC(name='auc')

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), metrics=metric)

model.fit(train_dataset, epochs=20, shuffle=True, validation_data=val_dataset, callbacks=callback, class_weight=class_weight)

gc.collect()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 128, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1/conv[0][0]                 
______________________________________________________________________________________________

Epoch 1/20
4576/4576 [==============================] - 460s 96ms/step - loss: 0.6618 - auc: 0.6366 - val_loss: 0.7320 - val_auc: 0.6615
Epoch 2/20
4576/4576 [==============================] - 453s 97ms/step - loss: 0.6270 - auc: 0.6991 - val_loss: 0.7058 - val_auc: 0.6747
Epoch 3/20
4576/4576 [==============================] - 445s 95ms/step - loss: 0.6055 - auc: 0.7293 - val_loss: 0.7492 - val_auc: 0.6808
Epoch 4/20
4576/4576 [==============================] - 449s 96ms/step - loss: 0.5868 - auc: 0.7522 - val_loss: 0.7766 - val_auc: 0.6836
Epoch 5/20
4576/4576 [==============================] - 449s 96ms/step - loss: 0.5665 - auc: 0.7742 - val_loss: 0.6721 - val_auc: 0.6851
Epoch 6/20
4576/4576 [==============================] - 454s 97ms/step - loss: 0.5448 - auc: 0.7954 - val_loss: 0.6245 - val_auc: 0.6830
Epoch 7/20
4576/4576 [==============================] - 445s 95ms/step - loss: 0.5191 - auc: 0.8185 - val_loss: 0.6091 - val_auc: 0.6826
Epoch 8/20
4576/4576 [===================

764

## test

In [11]:
record_file_val = 'copd_test_seg_new.tfrecords'
test_dataset = (tf.data.TFRecordDataset(
    record_file_val, buffer_size=256, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_test)
.batch(BATCH_SIZE))

y_test = get_data_label(split='test', category=None, types=0)

In [12]:
checkpoint_filepath = 'checkpoints/AUC/checkpoint_seg_Dnet121'
model = load_model_from_pretrain('Dnet121')

model.load_weights(checkpoint_filepath)

y_preds = model.predict(test_dataset)

In [13]:
# 1034
test_CI(y_preds, y_test)

print(average_precision_score(y_test, y_preds, average=None))

thresh = get_thresh(y_test, y_preds, 'Youden')
print('thresh:', thresh)

print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

thresh = get_thresh(y_test, y_preds, 'G-mean')
print('thresh:', thresh)

print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

thresh = 0.5
print('thresh:', thresh)

print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

gc.collect()

Sample auc mean: 0.69
Samole auc std: 0.00
Sample auc CI: 0.01
Confidence interval for AUC: [0.69 - 0.70]
0.3592227193585121
thresh: 0.44451392
              precision    recall  f1-score   support

           0       0.89      0.59      0.71     36808
           1       0.29      0.70      0.41      9026

    accuracy                           0.61     45834
   macro avg       0.59      0.64      0.56     45834
weighted avg       0.77      0.61      0.65     45834

thresh: 0.47661343
              precision    recall  f1-score   support

           0       0.88      0.63      0.74     36808
           1       0.30      0.65      0.41      9026

    accuracy                           0.64     45834
   macro avg       0.59      0.64      0.57     45834
weighted avg       0.77      0.64      0.67     45834

thresh: 0.5
              precision    recall  f1-score   support

           0       0.88      0.66      0.76     36808
           1       0.31      0.62      0.41      9026

    acc

87271